In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import albumentations as A
import albumentations.pytorch
import cv2
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from cutmix.cutmix import CutMix
from cutmix.utils import CutMixCrossEntropyLoss
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from tqdm import tqdm
from edafa import ClassPredictor
import wandb

config={
    "lr": 1e-3,
    "dropout": 0.5,
    "architecture": "efficientnet-b4",
    "dataset": "vanilla + mask",
    "augmentation" : "cutmix",
    "gamma" : 0.95,
    "batch_size" : 64,
    "epochs" : 20,
    "memo" : 'TTA'
}
config['model'] = '_'.join([config['architecture'], config['augmentation'], str(config['batch_size']), str(config['lr']), str(config['gamma']), config['memo']])
# wandb.init(project="MaskClassification", name=config['model'], config=config)



In [2]:
print('CUDA GPU available : {}'.format(torch.cuda.is_available()))
try:
    print('{} GPU(s) is(are) allocated'.format(torch.cuda.device_count()))
except:
    print('GPUs are not allocated. Current runtime is on CPU.')
device = torch.device("cuda")
CUDA_LAUNCH_BLOCKING=1

CUDA GPU available : True
1 GPU(s) is(are) allocated


In [3]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'
train_dir = '/opt/ml/input/data/train'


## Dataset

In [4]:
from torch.autograd import Variable

class TrainDataset(Dataset):
    def __init__(self, train_dir, is_Train=True, transform=None):
        super().__init__()
        
        csv_path = os.path.join(train_dir, 'train.csv')
        csv = pd.read_csv(csv_path)
        self.image_dir = os.path.join(train_dir, 'images')
        self.transform = transform
        self.image_path = []
        path = csv['path']
        
        for p in path:
            images = [os.path.join(*[self.image_dir, p, image]) for image in os.listdir(os.path.join(self.image_dir, p)) if not image[:1] == '.']
            for image in images:
                self.image_path.append(image)
                
        self.comb_dic = {}
        comb = [(m, g, a) for m in ['m', 'i', 'n'] for g in ['male', 'female'] for a in [0, 1, 2]]
        for i, (m, g, a) in enumerate(comb):
            self.comb_dic[(m, g, a)] = i
            
    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, idx):
        image_name = self.image_path[idx]
        image = cv2.imread(image_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        features = image_name.split('/')[-2:]
        
        mask = features[1][0]
        age = int(features[0].split('_')[-1])
        gender = features[0].split('_')[1]
        
        if age >= 57: # 원래 60
            age = 2
        elif age >= 30: # 원래 30
            age = 1
        else:
            age = 0        
        
        target = self.comb_dic[(mask, gender, age)]
        
        if self.transform:
            image = self.transform(image=image)['image']
        
        return image, target

class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = cv2.imread(self.img_paths[index])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image=image)['image']
        return image

    def __len__(self):
        return len(self.img_paths)

In [5]:
from torch.utils.data.sampler import SubsetRandomSampler
# from torch.utils.data.dataset import random_split

tfms = A.Compose([
        A.augmentations.crops.transforms.CenterCrop(400, 360, p=1.0),
        A.augmentations.geometric.resize.Resize(224, 224, interpolation=1, p=1),
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.6),
        A.RandomBrightnessContrast(brightness_limit=0.1, p=0.6),
        A.GaussNoise(p=0.5),
        A.transforms.Normalize(mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246), max_pixel_value=255.0, p=1.0),
        A.pytorch.transforms.ToTensorV2(),
    ])
tfms_test = A.Compose([
        A.augmentations.crops.transforms.CenterCrop(400, 360, p=1.0),
        A.augmentations.geometric.resize.Resize(224, 224, interpolation=1, p=1),
        A.transforms.Normalize(mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246), max_pixel_value=255.0, p=1.0),
        A.pytorch.transforms.ToTensorV2(),
    ])
val_idx = []
person_idx = list(range(2700))

np.random.seed(211)
np.random.shuffle(person_idx)

n_val_person = int(2700 * 0.2)
val_person_idx = person_idx[:n_val_person]
train_indices = []
val_indices = []
for i in range(2700*15):
    if i // 15 not in val_person_idx:
        train_indices.append(i)
    else:
        val_indices.append(i)

# print(train_indices)
# print(val_indices)
# train_sampler = SubsetRandomSampler(train_indices)
# valid_sampler = SubsetRandomSampler(val_indices)
np.random.shuffle(train_indices)
np.random.shuffle(val_indices)

dataset = TrainDataset(train_dir, transform=tfms)
dataset = CutMix(dataset, num_class=18, beta=1.0, prob=0.5, num_mix=2)
# print(len(dataset))
# plt.imshow(np.array(train_dataset[312][0]['image'].permute(1,2,0)))

train_loader = DataLoader(dataset, batch_size=config['batch_size'], num_workers=4, pin_memory=True, sampler=train_indices)
val_loader = DataLoader(dataset, num_workers=4, batch_size=config['batch_size'], pin_memory=True, sampler=val_indices)

In [6]:
# plt.imshow(np.array(train_dataset[312][0]['image'].permute(1,2,0)))
# dataset[0]

## Model

In [7]:
from efficientnet_pytorch import EfficientNet
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.backbone = EfficientNet.from_pretrained(config['architecture'], num_classes=18)
        self.best_f1 = 0
        self.best_valid = 999999999
    def forward(self, x):
        return self.backbone(x)
        
model = Net().to(device)


class myPredictor(ClassPredictor):
    def __init__(self,model,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.model = model
        self.trans = transforms.Compose([
    transforms.ToTensor()])

    def predict_patches(self,patches):
        preds = []
        for i in range(patches.shape[0]):
            processed = self.trans(patches[i])
            pred = self.model(processed)
            preds.append(pred.data.numpy())
        return self.model(patches)

p = myPredictor(model, "./TTA.json")

Loaded pretrained weights for efficientnet-b4


In [8]:
import torch.optim as optim 
                             
criterion = CutMixCrossEntropyLoss(True)

optimizer = optim.Adam(model.parameters(), lr=config['lr'])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=config['gamma'], verbose=True)

Adjusting learning rate of group 0 to 1.0000e-03.


In [9]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

def check():
    model.eval()
    val_loss = 0.0
    counter = 0
    y_true = []
    y_predicted = []
    
    with torch.no_grad():
        prog_bar = tqdm(enumerate(val_loader), total=int(len(val_indices)/val_loader.batch_size))
        for i, (x, y) in prog_bar:
            counter += 1
            y_true += y.tolist()
            
            x = x.to(device=device)
            y = y.to(device=device)
    
            scores = p.predict_images([x])
        
            
            _, predictions = scores.max(1)
            y_predicted += predictions.tolist()
            
            loss = criterion(scores, y)
            
            val_loss += loss.item()
#     print(y_true, y_predicted)
#     cm = confusion_matrix(y_true, y_predicted)
#     F1 = []
#     for c in range(18):
#         precision = cm[c][c] / np.sum(cm, axis=0)[c]
#         recall = cm[c][c] / np.sum(cm, axis=1)[c]
#         F1.append(2 * precision * recall / (precision + recall))
#     macro_F1 = np.mean(F1)
    
    print("< VALIDATION >")
    print("*"*73)
    print("Validation Loss :", val_loss/counter)
    print("-"*73)
#     print("Total Accuracy")
#     print(accuracy_score(y_true, y_predicted) * 100, "%")
#     print("-"*73)
#     print("Confusion Matrix")
#     for row in cm:
#         for c in row:
#             print(str(c).ljust(4), end='')
#         print()
#     print("-"*73)
#     print("Validation F1 score :" , macro_F1)
#     for c, f in enumerate(F1):
#         print("Class", c, ":", f)
        
#     if model.best_f1 < macro_F1:
#         model.best_f1 = macro_F1
    if model.best_valid > val_loss/counter:
        model.best_valid = val_loss/counter
        torch.save(model.state_dict(), '/opt/ml/weights/{}/{:.4f}.pt'.format(config['model'], val_loss/counter))
    print("model saved!")
    print("*"*73)
    print()
    wandb.log({
        "Validation Loss" : val_loss/counter, 
#         "Validation Total Accuracy" :accuracy_score(y_true, y_predicted) * 100, 
#         "Validation F1" : macro_F1,
    })
        
    model.train()

In [10]:


folder = '/opt/ml/weights/{}'.format(config['model'])
if not os.path.exists(folder):
    os.mkdir(folder)

for epoch in range(config['epochs']):
    print("Epoch :", epoch + 1)
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    total = 0
    total_it = int(len(train_indices)/train_loader.batch_size)
    prog_bar = tqdm(enumerate(train_loader), total=total_it)
    for i, (inputs, labels) in prog_bar:
        
        counter += 1
        optimizer.zero_grad()
        inputs = inputs.to(device)
        
        for x in inputs:
            x = x.unsqueeze(0).permute(0,2,3,1).detach().cpu().numpy()
            output = p.predict_images(x)
            print(output)
        labels = labels.to(device)
        total += labels.size(0)
    
        #######
        preds = []
        for img in inputs:
            print(img.shape)
            img = img[0,:,:,:]
            b, h, w, c = img.shape
            aug_patch = np.zeros((3,h,w,c),dtype=img.dtype)
            for i,aug in enumerate(self.augs):
                aug_patch[i] = apply(aug,img,self.bits)
            #return aug_patch
        print(np.array(preds).shape, np.array(preds))
        
        #######
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _, preds = torch.max(outputs.data, 1)
#         train_running_correct += (preds == labels).sum().item()
        
        train_running_loss += loss.item()
        
        if i == total_it//2 or i == total_it-1:
            train_loss = train_running_loss / counter
#             train_accuracy = 100. * train_running_correct / total
            
            print("Train Loss :", train_loss)
#             print("Train Accuracy :", train_accuracy)
            wandb.log({"Train Loss" : train_loss})
            check()
    scheduler.step()
    train_running_loss = 0.0
    train_running_correct = 0
print("Finish")

Epoch : 1


  0%|          | 0/506 [00:01<?, ?it/s]


TypeError: ToTensor() takes no arguments

## Testing

In [ ]:
# model.load_state_dict(torch.load('/opt/ml/weights/3way/{}/{:.4f}.pt'.format(config['model'], model.best_f1))), model.best_f1
model.load_state_dict(torch.load('/opt/ml/weights/{}/0.6784.pt'.format(config['model'])))

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
dataset = TestDataset(image_paths, tfms_test)

loader = DataLoader(
    dataset,
    shuffle=False,
    batch_size=64,
    num_workers=4
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
# device = torch.device('cuda')

model.eval()
device = torch.device("cuda:0")
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []

prog_bar = tqdm(enumerate(loader), total=int(len(dataset)/loader.batch_size))
for i, images in prog_bar:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

In [ ]:
idx = 1040
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

image = cv2.imread(image_paths[idx])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

image = tfms_test(image=image)['image'].to(device)
image = image.unsqueeze(0)
pred = model(image)
label = np.argmax(pred.detach().cpu().numpy())
#
masklabel = {0: "Mask", 1: "Incorrect", 2: "Normal"}
genderlabel = {0: "Male", 1: "Female"}
agelabel = {0: "~ 30", 1: "30 ~ 60", 2: "60 ~"}

feature_to_label = {}
features = [(m, g, a) for m in ['Mask', 'Incorrect', 'Normal'] for g in ['Male', 'Female'] for a in ["~ 30", "30 ~ 60", "60 ~"]]
for i, (m, g, a) in enumerate(features):
    feature_to_label[(m, g, a)] = i
#
label_to_feature = { feature_to_label[k]:k for k in feature_to_label}
m, g, a = label_to_feature[label]
print(m)
print(g)
print(a)
